In [1]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [2]:
fecha_hoy=datetime.now().date()
mes_hoy=fecha_hoy.month
año_hoy=fecha_hoy.year
fecha_hoy,mes_hoy,año_hoy

(datetime.date(2024, 11, 17), 11, 2024)

In [3]:
start_date='2024-01-01'
end_date='2024-10-31'
category='generacion'
widget_gen='estructura-generacion'
widget_pot='potencia-instalada'
time_trunc='month'
start_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.strptime(end_date, '%Y-%m-%d')
horas=((end_dt-start_dt).days+1)*24
horas_año=8760
horas_proporcional=horas/horas_año
horas_tec_teoricas={
    'Ciclo combinado' : 6000,
    'Nuclear' : 8760,
    'Solar fotovoltaica' : 2000,
    'Eólica' : 2200,
    'Hidráulica' : 4000,
    'Cogeneración' : 7000,
    'Turbinación bombeo' : 2000
}
#geo_trunc='electric_system' #no hay más disponibles

In [4]:
def download_redata(category,widget,start_date,end_date,time_trunc): #,geo_trunc):
    column_mapping = {
        "estructura-generacion": {"valor": "gen_TWh", "porcentaje": "porc_gen", "coef":1000000},
        "potencia-instalada": {"valor": "pot_GW", "porcentaje": "porc_pot", "coef": 1000}
    }
    
    end_point='https://apidatos.ree.es/es/datos'
    url=f'{end_point}/{category}/{widget}?start_date={start_date}T00:00&end_date={end_date}T23:59&time_trunc={time_trunc}'
    request=requests.get(url)
    data=request.json()
    datos = []
    for tech in data["included"]:
        tech_name = tech["attributes"]["title"]
        for entry in tech["attributes"]["values"]:
            datos.append({"mes": entry["datetime"], "tecnologia": tech_name, "valor": entry["value"],"porcentaje":entry["percentage"]})

    # Convertir a DataFrame
    df_in_gen = pd.DataFrame(datos)
    df_in_gen['mes']=pd.to_datetime(df_in_gen['mes'], utc=True).dt.tz_convert('Europe/Madrid').dt.tz_localize(None)
    df_in_gen['mes_num']=df_in_gen['mes'].dt.month
    coef = column_mapping[widget]["coef"]
    df_in_gen['valor']=df_in_gen['valor']/coef
    if widget in column_mapping:
        df_in_gen.rename(columns={
            "valor": column_mapping[widget]["valor"],
            "porcentaje": column_mapping[widget]["porcentaje"]
        }, inplace=True)
    
    return df_in_gen


In [5]:
df_in_gen=download_redata(category,widget_gen,start_date,end_date,time_trunc)

In [6]:
df_in_pot=download_redata(category,widget_pot,start_date,end_date,time_trunc)

In [7]:
df_in=pd.merge(df_in_gen,df_in_pot, on=['mes', 'mes_num','tecnologia'], how='outer',)
df_in['horas_gen']=round(df_in['gen_TWh']*1000/df_in['pot_GW'],1)
df_in['gen_TWh']=round(df_in['gen_TWh'],1)
df_in['pot_GW']=round(df_in['pot_GW'],1)

In [8]:
df_in

,mes,tecnologia,gen_TWh,porc_gen,mes_num,pot_GW,porc_pot,horas_gen
0,2024-01-01,Hidráulica,3.9,0.166650,1,17.1,0.136531,228.2
1,2024-02-01,Hidráulica,3.0,0.134581,2,17.1,0.136138,174.5
2,2024-03-01,Hidráulica,4.7,0.208092,3,17.1,0.135668,273.1
3,2024-04-01,Hidráulica,4.0,0.191213,4,17.1,0.134993,235.6
4,2024-05-01,Hidráulica,2.9,0.136865,5,17.1,0.134669,170.8
...,...,...,...,...,...,...,...,...
185,2024-06-01,Potencia total,NaN,NaN,6,127.4,1.000000,NaN
186,2024-07-01,Potencia total,NaN,NaN,7,128.6,1.000000,NaN
187,2024-08-01,Potencia total,NaN,NaN,8,128.9,1.000000,NaN
188,2024-09-01,Potencia total,NaN,NaN,9,129.1,1.000000,NaN


In [9]:
df_out_ratio=df_in.groupby('tecnologia').agg({
    'gen_TWh':'sum',
    'pot_GW':'last',
    'horas_gen':'sum'
})
df_out_ratio=df_out_ratio.reset_index()
#eliminamos las filas de gen y pot total
df_out_ratio=df_out_ratio[~df_out_ratio['tecnologia'].isin(['Generación total','Potencia total'])]

In [10]:
df_out_ratio

,tecnologia,gen_TWh,pot_GW,horas_gen
0,Carbón,2.3,2.1,1147.9
1,Ciclo combinado,26.1,26.3,989.0
2,Cogeneración,13.3,5.6,2373.3
3,Eólica,50.7,31.6,1622.3
4,Fuel + Gas,0.0,0.0,0.0
6,Hidroeólica,0.0,0.0,1894.4
7,Hidráulica,29.3,17.1,1710.2
8,Motores diésel,2.1,0.8,2705.1
9,Nuclear,44.2,7.1,6212.7
10,Otras renovables,3.1,1.1,2742.8


In [11]:
gen_total=round(df_out_ratio['gen_TWh'].sum(),1)
pot_total=round(df_out_ratio['pot_GW'].sum(),1)
gen_total,pot_total

(223.3, 129.1)

In [12]:

df_out_ratio

,tecnologia,gen_TWh,pot_GW,horas_gen
0,Carbón,2.3,2.1,1147.9
1,Ciclo combinado,26.1,26.3,989.0
2,Cogeneración,13.3,5.6,2373.3
3,Eólica,50.7,31.6,1622.3
4,Fuel + Gas,0.0,0.0,0.0
6,Hidroeólica,0.0,0.0,1894.4
7,Hidráulica,29.3,17.1,1710.2
8,Motores diésel,2.1,0.8,2705.1
9,Nuclear,44.2,7.1,6212.7
10,Otras renovables,3.1,1.1,2742.8


In [13]:
df_out_ratio['horas_gen']=df_out_ratio['horas_gen'].astype(int)
df_out_ratio['FC']=round(df_out_ratio['horas_gen']/horas,3)
df_out_ratio['%_mix']=round(df_out_ratio['gen_TWh']/gen_total,3)


In [14]:
df_out_ratio

,tecnologia,gen_TWh,pot_GW,horas_gen,FC,%_mix
0,Carbón,2.3,2.1,1147,0.157,0.010
1,Ciclo combinado,26.1,26.3,989,0.135,0.117
2,Cogeneración,13.3,5.6,2373,0.324,0.060
3,Eólica,50.7,31.6,1622,0.222,0.227
4,Fuel + Gas,0.0,0.0,0,0.000,0.000
6,Hidroeólica,0.0,0.0,1894,0.259,0.000
7,Hidráulica,29.3,17.1,1710,0.234,0.131
8,Motores diésel,2.1,0.8,2705,0.370,0.009
9,Nuclear,44.2,7.1,6212,0.849,0.198
10,Otras renovables,3.1,1.1,2742,0.375,0.014


In [15]:
#tecnologias seleccionadas para graficar
tec_select=['Ciclo combinado', 'Hidráulica', 'Nuclear', 'Solar fotovoltaica', 'Turbinación bombeo', 'Eólica', 'Cogeneración']
df_out_ratio_select=df_out_ratio[df_out_ratio['tecnologia'].isin(tec_select)].copy()
#colores asociados (lo pongo aquí para tener la relacion directa)
colores=["#555867","#4be4ff","#ff2b2b","#ff8700","#004280","#09ab3b","#6d3fc0"]




In [16]:
#calculamos las horas teoricas maximas
df_out_ratio_select['horas_max']=horas_proporcional*df_out_ratio_select['tecnologia'].map(horas_tec_teoricas)
df_out_ratio_select['horas_max']=df_out_ratio_select['horas_max'].astype(int)
#calculamos el FC relativo según horas máximas
df_out_ratio_select['FC_rel']=round(df_out_ratio_select['horas_gen']/df_out_ratio_select['horas_max'],3)

df_out_ratio_select['%_mix']=round(df_out_ratio_select['gen_TWh']/gen_total,3)

In [17]:
#ordenamos la tabla por FC
df_out_ratio_select_fc=df_out_ratio_select.sort_values(['FC'],ascending=False)
df_out_ratio_select_fcrel=df_out_ratio_select.sort_values(['FC_rel'],ascending=False)
df_out_ratio_select_mix=df_out_ratio_select.sort_values(['%_mix'],ascending=False)

In [18]:
df_out_ratio_select_fc

,tecnologia,gen_TWh,pot_GW,horas_gen,FC,%_mix,horas_max,FC_rel
9,Nuclear,44.2,7.1,6212,0.849,0.198,7320,0.849
2,Cogeneración,13.3,5.6,2373,0.324,0.060,5849,0.406
7,Hidráulica,29.3,17.1,1710,0.234,0.131,3342,0.512
3,Eólica,50.7,31.6,1622,0.222,0.227,1838,0.882
18,Turbinación bombeo,4.8,3.3,1449,0.198,0.021,1671,0.867
14,Solar fotovoltaica,39.7,29.6,1402,0.192,0.178,1671,0.839
1,Ciclo combinado,26.1,26.3,989,0.135,0.117,5013,0.197


In [19]:
df_out_ratio_select_fcrel

,tecnologia,gen_TWh,pot_GW,horas_gen,FC,%_mix,horas_max,FC_rel
3,Eólica,50.7,31.6,1622,0.222,0.227,1838,0.882
18,Turbinación bombeo,4.8,3.3,1449,0.198,0.021,1671,0.867
9,Nuclear,44.2,7.1,6212,0.849,0.198,7320,0.849
14,Solar fotovoltaica,39.7,29.6,1402,0.192,0.178,1671,0.839
7,Hidráulica,29.3,17.1,1710,0.234,0.131,3342,0.512
2,Cogeneración,13.3,5.6,2373,0.324,0.060,5849,0.406
1,Ciclo combinado,26.1,26.3,989,0.135,0.117,5013,0.197


In [20]:
df_out_ratio_select_mix

,tecnologia,gen_TWh,pot_GW,horas_gen,FC,%_mix,horas_max,FC_rel
3,Eólica,50.7,31.6,1622,0.222,0.227,1838,0.882
9,Nuclear,44.2,7.1,6212,0.849,0.198,7320,0.849
14,Solar fotovoltaica,39.7,29.6,1402,0.192,0.178,1671,0.839
7,Hidráulica,29.3,17.1,1710,0.234,0.131,3342,0.512
1,Ciclo combinado,26.1,26.3,989,0.135,0.117,5013,0.197
2,Cogeneración,13.3,5.6,2373,0.324,0.060,5849,0.406
18,Turbinación bombeo,4.8,3.3,1449,0.198,0.021,1671,0.867


In [31]:
mix_tec_select=df_out_ratio_select_mix['%_mix'].sum()
mix_resto=round(1-mix_tec_select,3)
gen_resto=round(gen_total-df_out_ratio_select_mix['gen_TWh'].sum(),1)
pot_resto=pot_total-df_out_ratio_select_mix['pot_GW'].sum()
mix_tec_select,mix_resto,gen_resto
nueva_fila = {
    'tecnologia': 'Resto',
    'gen_TWh': gen_resto,
    'pot_GW': pot_resto,  # Opcional: si no aplica, puedes dejar como None
    'horas_gen': None,
    'FC': None,
    '%_mix': mix_resto,
    'horas_max': None,
    'FC_rel': None
}
print(pot_total,pot_resto)
df_out_ratio_select_mix=pd.concat([df_out_ratio_select_mix,pd.DataFrame([nueva_fila])],ignore_index=True)
df_out_ratio_select_mix=df_out_ratio_select_mix.sort_values(['%_mix'],ascending=False)

129.1 8.5


In [32]:
df_out_ratio_select_mix

,tecnologia,gen_TWh,pot_GW,horas_gen,FC,%_mix,horas_max,FC_rel
0,Eólica,50.7,31.6,1622,0.222,0.227,1838,0.882
1,Nuclear,44.2,7.1,6212,0.849,0.198,7320,0.849
2,Solar fotovoltaica,39.7,29.6,1402,0.192,0.178,1671,0.839
3,Hidráulica,29.3,17.1,1710,0.234,0.131,3342,0.512
4,Ciclo combinado,26.1,26.3,989,0.135,0.117,5013,0.197
7,Resto,15.2,8.5,None,None,0.068,None,None
5,Cogeneración,13.3,5.6,2373,0.324,0.060,5849,0.406
6,Turbinación bombeo,4.8,3.3,1449,0.198,0.021,1671,0.867


In [33]:
#creamos un diccionario de colores asociados a cada tecnología
colores_tecnologia = {tec: colores[i % len(colores)] for i, tec in enumerate(tec_select)}

In [34]:
colores_tecnologia

{'Ciclo combinado': '#555867',
 'Hidráulica': '#4be4ff',
 'Nuclear': '#ff2b2b',
 'Solar fotovoltaica': '#ff8700',
 'Turbinación bombeo': '#004280',
 'Eólica': '#09ab3b',
 'Cogeneración': '#6d3fc0'}

In [35]:
graf_bolas=px.scatter(df_out_ratio_select_fc,x='pot_GW',y='gen_TWh',size='horas_gen', 
                    size_max=100, color=df_out_ratio_select_fc['tecnologia'], 
                    hover_name=df_out_ratio_select_fc['tecnologia'],
                    color_discrete_map=colores_tecnologia,
                    width=1300,
                    hover_data={
                        'tecnologia':False,
                        'FC':True
                    }
                    )
graf_bolas.update_traces(
    text=df_out_ratio_select_fc['tecnologia'],  # Usa los índices (tecnologías) como texto
    textposition='middle center',  # Coloca el texto en el centro de las burbujas
    #textfont=dict(size=12, color="black"),  # Tamaño y color del texto
    )
graf_bolas.update_layout(
    title=dict(
        text='Potencia instalada, generación y sus horas equivalentes',
        x=.5,
        xanchor='center'

    )

)

In [36]:
graf_FC=px.bar(df_out_ratio_select_fc,x='FC',y='tecnologia',
                    orientation='h',
                    color=df_out_ratio_select_fc['tecnologia'], 
                    hover_name=df_out_ratio_select_fc['tecnologia'],
                    color_discrete_map=colores_tecnologia,
                    width=1300,
                    text_auto=True,
                    hover_data={
                        'tecnologia':False,
                        'horas_gen':True
                    },
                    text='FC',
                    
                    )
graf_FC.update_traces(
    texttemplate='%{text:.1%}',
    textposition='inside',  # Coloca el texto en el centro de las burbujas
    #textfont=dict(size=12, color="black"),  # Tamaño y color del texto
    )
graf_FC.update_layout(
    title=dict(
        text='Factor de carga (%)',
        x=.5,
        xanchor='center',
    ),
    xaxis_tickformat='.0%',
    bargap=.4
)


In [37]:

graf_FC_rel=px.bar(df_out_ratio_select_fcrel,x='FC_rel',y='tecnologia',
                    orientation='h',
                    color=df_out_ratio_select_fcrel['tecnologia'], 
                    hover_name=df_out_ratio_select_fcrel['tecnologia'],
                    color_discrete_map=colores_tecnologia,
                    width=1300,
                    text_auto=True,
                    hover_data={
                        'tecnologia':False,
                        'horas_max':True
                    },
                    text='FC_rel'
                    
                    )
graf_FC_rel.update_traces(
    texttemplate='%{text:.1%}',
    textposition='inside',  # Coloca el texto en el centro de las burbujas
    #textfont=dict(size=12, color="black"),  # Tamaño y color del texto
    )
graf_FC_rel.update_layout(
    title=dict(
        text='Factor de carga según horas máximas (%)',
        x=.5,
        xanchor='center',
    ),
    xaxis_tickformat='.0%',
    bargap=.4
)

In [ ]:
graf_FC_mix=px.bar(df_out_ratio_select_mix,x='%_mix',y='tecnologia',
                    orientation='h',
                    color=df_out_ratio_select_mix['tecnologia'], 
                    hover_name=df_out_ratio_select_mix['tecnologia'],
                    color_discrete_map=colores_tecnologia,
                    width=1300,
                    text_auto=True,
                    hover_data={
                        'tecnologia':False,
                        'gen_TWh':True,
                        'pot_GW':True
                    },
                    text='%_mix'
                    
                    )
graf_FC_mix.update_traces(
    #formateamos el texto de las barras
    texttemplate='%{text:.1%}',
    textposition='inside',  # Coloca el texto en el centro de las burbujas
    #textfont=dict(size=12, color="black"),  # Tamaño y color del texto
    )
graf_FC_mix.update_layout(
    title=dict(
        text='Aportación al mix de generación (%)',
        x=.5,
        xanchor='center',
        
    ),
    xaxis_tickformat='.0%',
    bargap=.4
)

In [60]:
df_out_ratio_select_mix

,tecnologia,gen_TWh,pot_GW,horas_gen,FC,%_mix,horas_max,FC_rel
0,Eólica,50.7,31.6,1622,0.222,0.227,1838,0.882
1,Nuclear,44.2,7.1,6212,0.849,0.198,7320,0.849
2,Solar fotovoltaica,39.7,29.6,1402,0.192,0.178,1671,0.839
3,Hidráulica,29.3,17.1,1710,0.234,0.131,3342,0.512
4,Ciclo combinado,26.1,26.3,989,0.135,0.117,5013,0.197
7,Resto,15.2,8.5,None,None,0.068,None,None
5,Cogeneración,13.3,5.6,2373,0.324,0.060,5849,0.406
6,Turbinación bombeo,4.8,3.3,1449,0.198,0.021,1671,0.867


In [67]:
colores_tecnologia['Resto']= '#D3D3D3'

In [72]:
graf_mix_queso=px.pie(df_out_ratio_select_mix, names='tecnologia', values='%_mix',
                    color='tecnologia',
                    color_discrete_map=colores_tecnologia,
                    hover_name='tecnologia',
                                      
                    hole=.4
                    )

graf_mix_queso.update_traces(textinfo='percent+label')


graf_mix_queso

In [73]:
graf_FC_bis=go.Figure()

graf_FC_bis.add_trace(go.Bar(
                    
                    x=df_out_ratio_select['FC_rel'],
                    y=df_out_ratio_select['tecnologia'],
                    orientation='h',
                    marker=dict(color=colores),
                        #color_discrete_map=colores_tecnologia,
                     
                    #hover_name=df_out_ratio_select['tecnologia'],
                    
                    #width=1300,
                    #text_auto=True,
                    width=.4,
                    opacity=.7                    
                    ))
graf_FC_bis.add_trace(go.Bar(
                    
                    x=df_out_ratio_select['FC'],
                    y=df_out_ratio_select['tecnologia'],
                    orientation='h',
                    marker=dict(color=colores),
                    width=.8,
                    opacity=1
                    #width=1300,
                    #text_auto=True,
                    
                    #   barmode='overlay'
                    ))



graf_FC_bis.update_traces(
    #text=df_out_ratio_select.index,  # Usa los índices (tecnologías) como texto
    textposition='inside',  # Coloca el texto en el centro de las burbujas
    #textfont=dict(size=12, color="black"),  # Tamaño y color del texto
    
    )
graf_FC_bis.update_layout(
    title=dict(
        text='Factor de carga (%)',
        x=.5,
        xanchor='center',
    ),
    bargap=.7,
    barmode='overlay'
)